In [10]:
import pandas, numpy as np,re
import xml, itertools, re, xml.etree.ElementTree as ET
from html.parser import HTMLParser
import datetime, tqdm
pars =HTMLParser()

def prettyPrintXML(s):
    r = xml.dom.minidom.parseString(s) #r.RegistroXML)
    print(r.toprettyxml())
    
from datetime import datetime as dt
import locale
locale.setlocale(locale.LC_ALL, 'es_es')
spanishDateToPandas = lambda s: pandas.to_datetime(dt.strptime(s, '%d %b %Y %H:%M:%S'))

# Data reading and basic cleaning

- Find candidates of repeated pateints
- Find the used codes for diagnosis and procedures

In [2]:
casos = pandas.read_csv('Venezolanas/casos.csv', index_col = 0)
pacientes = pandas.read_csv('Venezolanas/pacientes.csv', index_col = 0)
registros = pandas.read_csv('Venezolanas/registros.csv', index_col = 0)
diagnosis = pandas.read_csv('Venezolanas/diagnosis.csv', index_col = 0)
procedimientos = pandas.read_csv('Venezolanas/procedimientos.csv', index_col = 0)
procedimientosDesc = pandas.read_csv('Venezolanas/procedimientosID.csv', index_col = 0)

str2Date = lambda s:datetime.datetime.strptime(s, '%Y-%m-%d %H:%M:%S.%f') if isinstance(s, str) \
            else datetime.datetime(1, 1, 1)
registros.FechaInicioAtencion = registros.FechaInicioAtencion.map(pandas.to_datetime) #registros.FechaInicioAtencion.map(str2Date)
procedimientos.FechaDescripcion  = procedimientos.FechaDescripcion.map(pandas.to_datetime)
from html.parser import HTMLParser
pars =HTMLParser()
#pars.unescape

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,27,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
procedureIDFound = set()
for _, p in procedimientos.iterrows():
    et = xml.etree.ElementTree.fromstring(p.XmlDescripcion)
    s = re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', p.XmlDescripcion)
    for ss in s:
        procedureIDFound.add(ss)


In [4]:
procedimientosDesc.loc[sorted(procedureIDFound)].to_excel('procedimientosFiltered.xlsx')
procedimientosDesc.loc[sorted(procedureIDFound)].to_csv('procedimientosFiltered.csv')

In [5]:
unique = registros.Diagnostico.unique()
unique = [u for u in unique if isinstance(u, str)]
for u in unique:
    if u.upper() not in diagnosis.index:
        print(u)

0800


In [6]:
#Check duplicates by birthData
def candidateToRepeated(p1, p2):
    """
    Selects as candidate if they were born in the same name:
    - they have the same 1st surname
    - [TODO] they have similar height (~2cm), as found in the cases
    
    TODO: beware the case of twins!
    """
    #First surname criterio
    if p1[1].Ape1Afil.strip().lower() == p2[1].Ape1Afil.strip().lower():
        return True
    #TODO: Search height and see it's similar
    if False:
        return True
    #Otherwise return False
    return False
    

#Beware
candidates = []
threshold = '2010'
for c in pacientes.groupby('FechaNac'):
    #Arbitrary threshold for separating between mothers and borns
    if len(c[1]) >= 2 and c[0][:4] < threshold:
        for p1, p2 in itertools.combinations(c[1].iterrows(), r = 2):
            if p1[0] != p2[0] and candidateToRepeated(p1, p2):
                candidates.append([p1[0], p2[0]])
pandas.DataFrame.from_records(candidates).to_csv('repeatedCandidates.csv')

# Select the cases by procedure

- Based on the procedure, select the history register associated.
- TODO: Check that diagnosis and procedure are compatible

In [7]:
registersByPatient = registros.groupby('NumeroHistoria')

In [50]:
getprocID = lambda s:  re.findall('<idProcedimiento>([a-zA-Z0-9]*)</idProcedimiento>', s)[0]
getprocDate = lambda s:  re.findall('<fechaCirugia>(.*)</fechaCirugia>', s)[0]

def associateCaseAndHistory(interventionId, intervention, registersByPatient, 
                            nDaysThreshold = 3,  nDaysUpperThreshold = 0):
    pId = intervention.IdPaciente
    df = registersByPatient.get_group(pId)
    #interventionDate = spanishDateToPandas(getprocDate(intervention.XmlDescripcion))
    interventionDate = pandas.to_datetime(intervention.FechaRegistro)
    df = df.loc[df.FechaInicioAtencion.between(interventionDate - pandas.Timedelta(nDaysThreshold, 'D'),
                                      interventionDate +  pandas.Timedelta(nDaysUpperThreshold, 'D'))]
    #Do not count Triage in the cases ID
    caseIds =  [s for s in df.Caso.unique() if not s.startswith('TRI')]
    if len(caseIds) == 0:
        print('Error in intervention %d found %d. Code = %s' % (interventionId, df.Caso.nunique(),getprocID(i.XmlDescripcion) ))
        print(intervention.FechaDescripcion, intervention.FechaRegistro, interventionDate)
        return False
    else:
        return df.loc[df.FechaInicioAtencion == df.FechaInicioAtencion.max()].Caso.values[0]

In [51]:
#Identify borns and abortions using the proc ID
interventionToCase = {}
for k, i in tqdm.tqdm_notebook(procedimientos.iterrows()):
    interventionToCase[k] = (associateCaseAndHistory(k, i, registersByPatient), i.IdPaciente)


Error in intervention 46665 found 0. Code = H3094
2018-08-06 12:39:00 2018-08-06 12:42:43.797 2018-08-06 12:42:43.797000
Error in intervention 50673 found 0. Code = H3094
2018-12-17 09:53:00 2018-12-17 09:56:33.900 2018-12-17 09:56:33.900000
Error in intervention 53734 found 0. Code = H3094
2019-04-08 09:04:00 2019-04-08 09:15:26.037 2019-04-08 09:15:26.037000
Error in intervention 54099 found 0. Code = H2880
2019-04-22 08:53:00 2019-04-22 08:55:01.700 2019-04-22 08:55:01.700000
Error in intervention 54108 found 0. Code = H3094
2019-04-22 08:53:00 2019-04-22 11:50:38.920 2019-04-22 11:50:38.920000
Error in intervention 54232 found 0. Code = H3094
2019-04-26 11:47:00 2019-04-26 11:48:37.893 2019-04-26 11:48:37.893000
Error in intervention 54948 found 0. Code = H3094
2019-05-22 04:25:00 2019-05-22 16:27:39.220 2019-05-22 16:27:39.220000
Error in intervention 54969 found 0. Code = H3100
2019-05-23 01:27:00 2019-05-23 13:49:29.310 2019-05-23 13:49:29.310000
Error in intervention 55144 foun

In [39]:
errors = 0
for k, (i, _) in interventionToCase.items():
    if i is False:
        errors += 1
print('Nerrors = %d' % errors)

Nerrors = 46


# Parse information from cases
Now we need to parse the information from the cases

In [43]:
def getEpicrisis(casoID, registrosByCase):
    return registrosByCase[(registrosByCase.Caso == casoID) & 
                     (registrosByCase.Asunto.str.contains('Epicrisis', case = False) == True)]

In [56]:
errorsEpicrisis = 0
for k, (case, pId) in tqdm.tqdm_notebook(interventionToCase.items()):
    if case is False:
        continue
    elif  len(getEpicrisis(case, registersByPatient.get_group(pId))):
        pass
    else:
        errorsEpicrisis += 1
        

In [ ]:
#TODO: check that operation and diagnosis are compatible.

In [18]:
getprocID(i.XmlDescripcion)
for k,c in interventionToCase[k].iterrows():
    prettyPrintXML(c.RegistroXML)

<?xml version="1.0" ?>
<C Asunto="Triage" Caso="TRI232384" CentroA="01" Diagnostico="R102" IdPaciente="209748" Padre="NULL" PlanAdm="100" Prestador="8865876" Raiz="NULL" Registro="25">
	<row NombreCampo="PlanAdministradora" NombreTabla="CamposTexto" ValorCampo="100"/>
	<row NombreCampo="Prioridad" NombreTabla="CamposTexto" ValorCampo="III"/>
	<row NombreCampo="MotivoConsulta" NombreTabla="CamposTexto" ValorCampo="&quot;ESTOY BOTANDO LIQUIDOS Y TENGO DOLORES&quot;
PACIENTE DE 23 AÑOS DE EDAD QUE CONSULTA POR UN CUADRO CLINICO DE 2 HORAS DE EVOLUCION CONSISTENTE EN DOLOR TIPO CONTRACCION EN HIPOGASTRIO QUE SE IRRADIA A REGION LUMBOSACRA, ACOMPAÑADO DE SALIDA DE LIQUIDOS POR GENITALES. NO ESTA ASOCIADO A OTRA SINTOMATOLOGIA. REFIERE MOVIMIENTOS FETALES PRESENTES"/>
	<row NombreCampo="Valoracion" NombreTabla="CamposTexto" ValorCampo="Alerta"/>
	<row NombreCampo="Procedencia" NombreTabla="CamposTexto" ValorCampo="Bolívar-Cartagena"/>
	<row NombreCampo="FrecuenciaCardiaca" NombreTabla="Campo

In [406]:
d = getEpicrisis('AD152124', casos)

In [19]:
import xml.etree.ElementTree as ET


In [78]:
def getDataFromCase(case):
    """
    Get some information from the case
    """
    data = {}
    r = ET.fromstring(case.RegistroXML)
    try:
        data['FC']=r.find('.//row[@NombreCampo="FrecuenciaCardiaca"]').get('ValorCampo')
        data['FR']=r.find('.//row[@NombreCampo="FrecuenciaRespiratoria"]').get('ValorCampo')
        data['Presion']=r.find('.//row[@NombreCampo="Presion"]').get('ValorCampo')
        data['Temperatura']=r.find('.//row[@NombreCampo="Temperatura"]').get('ValorCampo')
        data['Talla']=r.find('.//row[@NombreCampo="Talla"]').get('ValorCampo')
        data['Peso']=r.find('.//row[@NombreCampo="Peso"]').get('ValorCampo')
        return data
    except:
        return {}

In [381]:
floatParse = '[0-9]*\.?[0-9]+'
compareGPCA = lambda s1, s2: all([i == j or not i or not j for i, j in zip(s1, s2)])
allEqual = lambda s: all([compareGPCA(i, j) for i, j in  itertools.combinations(s, r = 2)])
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGPAC = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGPAC += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGPAC += '|'
patternGPAC += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGPAC += '|' # C and A can be missing, or in other order
patternGPAC += ')'
searchGCPAC = re.compile(patternGPAC, flags=0)

meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SEPTIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
meses = meses + list(map(lambda s: s[:3], meses))
noRecuerda = ['NO', '\?']
separadorFecha = '(?:[\.\\/-]|DE)?'
date = '(([0-9]+)'+ sep + separadorFecha + sep + '([0-9]+|%s)'%  '|'.join(meses) + 
                       sep + separadorFecha + sep + '([0-9]+)

searchFUM = re.compile('FUM'+ sep + '(?::|.)?'+ sep + date + '|%s)' % ('|'.join(noRecuerda)), flags = re.IGNORECASE)
errors = []
def parseGPCA_and_fum(s):
    
    r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML",ET.fromstring(s).findall('row')))
    text = pars.unescape(r[0].attrib['ValorCampo'])
    
    allowedStarts = ['7-&gt;', '-', '- ANTECEDENTES']
    line = re.findall('^%s(?:%s)?%s' % (sep, '|'.join(allowedStarts), sep) + 'G' + sep + '[0-9]+.*$', text, re.M)
    if len(line) != 1:
        #If it doesn't find it, search brutally
        search = searchGCPCA.findall(s)
        searchCleaned = [(s[0], s[1], s[2] + s[-1], s[3] + s[-2]) for s in search]
        if search and allEqual(search):
            GPCA_OK = True
            parsedGPCA = search[0]
        else:
            GPCA_OK = False
            parsedGPCA = []

    else:
        f = lambda s: s[0] if s else ''
        G = f(re.findall('G' + sep + '([0-9]+)', line[0]))
        A = f(re.findall('A' + sep + '([0-9]+)', line[0]))
        C = f(re.findall('C' + sep + '([0-9]+)', line[0]))
        P = f(re.findall('P' + sep + '([0-9]+)', line[0])) 
        parsedGPCA = [G, P, A, C]
        GPCA_OK = True
    #Prob athere is a better way...
    parsedFUM = searchFUM.findall(s)
    return {'fum' : len(parsedFUM) > 0,
            'fum_OK' : len(parsedFUM) ,
            'GPCA_OK' : GPCA_OK,
            'fum_Data' : parsedFUM,
            'parsedGPCA' : parsedGPCA}

In [ ]:
def parseGestationalAge(s):
    pass

In [382]:
import tqdm
casos_wo_epi = 0
casos_several_epi = 0
results ={}
for i,c in tqdm.tqdm_notebook(enumerate(registros.groupby('Caso')), total = len(casos)):
    if i == 1000:
        break
    epicrisis= getEpicrisis(c[0], c[1])
    if len(epicrisis) == 0:    
        casos_wo_epi += 1
    elif len(epicrisis) >= 2:    
        casos_several_epi += 1
    else:
        stringRegistro = epicrisis.RegistroXML.iloc[0]
        results[c[0]] = parseGPCA_and_fum(stringRegistro)
        data = {}
        for cc in c[1].iterrows():
            data = getDataFromCase(cc[1])
            if data:
                break
        results[c[0]].update(data)
    #I am not looking at echo information
    #Search also for "ultimo parto"
    #Search for height, weight and vital signs (can they change?) in the rest of the report. They are a row field in HTML.

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


In [383]:
print('With epi: %d' % len(results))
print('Missing epi: %d' % casos_wo_epi)
print('Several epi: %d' % casos_several_epi)

With epi: 503
Missing epi: 440
Several epi: 57


In [384]:
pd = pandas.DataFrame.from_dict(results, orient = 'index')
np.sum(pd.fum_OK != 0), np.sum(pd.fum), np.sum(pd.GPCA_OK), len(pd) 

(467, 467, 486, 503)

In [385]:
pd[pd.fum_OK == False]

,fum,fum_OK,GPCA_OK,fum_Data,parsedGPCA,FC,FR,Presion,Temperatura,Talla,Peso
AD257917,False,0,True,[],"[2, 1, 1, 0]",80,20,110/60,37,1.60,49.5
AD275522,False,0,True,[],"[2, 2, 0, 2]",98,20,110/70,38.2,1.56,61
AD279691,False,0,True,[],"[4, 4, 0, 0]",105,20,110/62,37,1.60,70
AD280261,False,0,True,[],"[4, 4, 0, 0]",115,20,80/60,36,1.60,70
AD281778,False,0,True,[],"[1, 1, 0, 1]",86,16,110/70,36.5,1.55,55
AD284225,False,0,False,[],[],NaN,NaN,NaN,NaN,NaN,NaN
AD289421,False,0,True,[],"[4, 4, 0, 0]",76,18,80/60,37,1.6,74
AD293576,False,0,True,[],"[2, 0, 1, 0]",78,16,100/70,36.7,1.70,75
AD294448,False,0,True,[],"[2, 1, , ]",75,18,129/71,36,1.60,65
AD295710,False,0,True,[],"[2, 2, 0, 0]",138,30,90/55,36.7,1.52,70


In [392]:
epicrisis= getEpicrisis('AD321568', registros)
stringRegistro = epicrisis.RegistroXML.iloc[0]
parseGPCA_and_fum(stringRegistro)

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.


{'fum': False,
 'fum_OK': 0,
 'GPCA_OK': True,
 'fum_Data': [],
 'parsedGPCA': ['2', '1', '0', '0']}

In [393]:
allRegisters = registros[registros.Caso == 'AD257917']
epicrisis= getEpicrisis('AD257917', registros[registros.Caso == 'AD257917'])
s = epicrisis.RegistroXML.iloc[0]
l = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML",ET.fromstring(s).findall('row')))[0]
text = pars.unescape(l.attrib['ValorCampo'])
text

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


'- PERSONALES:\r\n  1-&gt; NIEGA\r\n\r\n  2-&gt; NIEGA\r\n\r\n  3-&gt; NIEGA\r\n\r\n  4-&gt; NIEGA\r\n\r\n  5-&gt; KETOTIFENO\r\n\r\n  6-&gt; NIEGA\r\n\r\n  7-&gt; MENARQUIA: 14 AÑOS.\r\nVIDA MARITAL: 16 AÑOS.\r\nVIDA OBSTETRICA: 17 AÑOS.\r\nG2 P1 C0 A1\r\n- FAMILIARES:\r\n  MADRE ASMA\r\n- ALÉRGICOS:\r\n  \r\n- QUIRÚRGICOS:'

In [349]:
prettyPrintXML(s) 

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Notas de Ingreso a Piso" Caso="AD333452" CentroA="01" CodigoDiagnosticoRelacionado1="Z370" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="Z988" IdPaciente="200822" Modulo="01" Padre="AD333452" PlanAdm="100" Prestador="9085835" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- DESCRIPCI&amp;#211;N DE LA NOTA:
PACIENTE DE 22 A&amp;#209;OS DE EDAD, CON DIAGNOSTICO DE:1. POP DE CESAREA SEGMENTARIA TRANSPERITONEAL (04/04/19 05:40 PM) 2. RECIEN NACIDO UNICO VIVO SEXO MASCULINO P:2480 T: 48CM 3. PUERPERIO INMEDIATO4. PRECLAMPSIA SEVERA5. CESAREA PREVIA 19-05-2008.
SUBJETIVO: PACIENTE REFIERE SENTIRSE BIEN, CON LEVE DOLOR EN REGION DE HERIDA QUIRURGICA, REFIERE SENSACIONES DE NAUSEAS, NIEGA CEFALEA, VOMITOS U OTRA SINTOMATOLOGIA ASOCIADA. 
OBJETIVO:SIGNOS VITALES: FC 82 LPM, FR 

In [140]:
sep= '\s*[,;:]?\s*'
 #In some cases there is only G
patternGCPCA = 'G%s(?P<G>[0-9]+)' %sep + sep + 'P%s(?P<P>[0-9]+)'  %sep + sep + '(?:'
patternGCPCA += 'A%s(?P<A>[0-9]+)' %sep  + sep + 'C%s(?P<C>[0-9]+)'  %sep
patternGCPCA += '|'
patternGCPCA += 'C%s(?P<C2>[0-9]+)' %sep  + sep + 'A%s(?P<A2>[0-9]+)' %sep
patternGCPCA += '|' # C and A can be missing, or in other order
patternGCPCA += ')'
searchGCPCA = re.compile(patternGCPCA, flags=0)
searchGCPCA.findall(epicrisis.iloc[0].RegistroXML)

[]

In [184]:
import html
xmlString = epicrisis.iloc[0].RegistroXML

r = list(filter(lambda S: S.attrib ['NombreCampo'] == "AntecedentesHTML",ET.fromstring(xmlString).findall('row')))
text = pars.unescape(r[0].attrib['ValorCampo'])
line = re.findall('^.*G' + sep + '[0-9]+.*$', text, re.M)
if len(line) != 2:
    #return False
    pass
G = re.findall('G' + sep + '([0-9]+)', line[0])[0]
A = re.findall('A' + sep + '([0-9]+)', line[0])[0]
C = re.findall('C' + sep + '([0-9]+)', line[0])[0]
P = re.findall('P' + sep + '([0-9]+)', line[0])[0] 

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  """


In [185]:
G

['2']

In [167]:
text.attrib['ValorCampo']

'- PERSONALES:\r\n  7-&amp;gt; -MENARQUIA:  14 A&#209;OS \r\n-VIDA MARITAL: 16 A&#209;OS\r\n-VODA OBSTETRICA: 16 A&#209;OS \r\n-G:2 A:0 P:1  C:0 \r\n-FUM: 12/08/17 EMBARAZO PARA HOY  DE 42.2 SEMANAS \r\n-CICLOS: IRREGULARES\r\n-PLANIFICACI&#211;N FAMILIAR: SI \r\n-GRUPO SANGUINEO: A+\r\n-COMPA&#209;EROS SEXUALES: 1\r\n-CONTROLES PRENATALES:2\r\n\r\nECOGRAFIA 6/01/2018 EMBARAZO DE 21 SEMANAS PARA HOY 40.4 SEMANAS\r\nECOGRAFIA 3/05/2018 EMBARAZO DE 37 SEMANAS PARA HOY 39.4 SEMANAS \r\nHEMOGRAMA 03/05/2018  HB 10.8 HTC. 34.1 PLAQUETA.299 LEU.22.8 \r\nVIH 02/05/2018 NEGATIVO\r\n- FAMILIARES:\r\n  SIN DATOS\r\n- AL&#201;RGICOS:\r\n  \r\n- QUIR&#218;RGICOS:'

In [141]:
print(epicrisis.iloc[0].RegistroXML)
searchGCPCA.findall(epicrisis.iloc[0].RegistroXML)

<C IdPaciente="183696" Caso="AD290433" Registro="450" Asunto="Epicrisis de la Historia Ingreso de Urgencias" Raiz="01" Padre="AD290433" PlanAdm="100" Prestador="9068204" Modulo="01" Diagnostico="O623" CodigoDiagnosticoRelacionado1="R102" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" CentroA="01"><row NombreCampo="Antecedentes" ValorCampo="" NombreTabla="CamposTexto"/><row NombreCampo="AntecedentesHTML" ValorCampo="- PERSONALES:&#x0D;&#x0A;  7-&amp;amp;gt; -MENARQUIA:  14 A&amp;#209;OS &#x0D;&#x0A;-VIDA MARITAL: 16 A&amp;#209;OS&#x0D;&#x0A;-VODA OBSTETRICA: 16 A&amp;#209;OS &#x0D;&#x0A;-G:2 A:0 P:1  C:0 &#x0D;&#x0A;-FUM: 12/08/17 EMBARAZO PARA HOY  DE 42.2 SEMANAS &#x0D;&#x0A;-CICLOS: IRREGULARES&#x0D;&#x0A;-PLANIFICACI&amp;#211;N FAMILIAR: SI &#x0D;&#x0A;-GRUPO SANGUINEO: A+&#x0D;&#x0A;-COMPA&amp;#209;EROS SEXUALES: 1&#x0D;&#x0A;-CONTROLES PRENATALES:2&#x0D;&#x0A;&#x0D;&#x0A;ECOGRAFIA 6/01/2018 EMBARAZO DE 21 SEMANAS PARA HOY 40.4 SEMANAS&#x0D;&#x0A;ECOGRAFIA 3/05/2

[]

KeyError: 1

In [48]:
for a in allRegisters.iterrows():
    print(prettyPrintXML(a[1].RegistroXML))

C:\Users\CMRC\Miniconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  


<?xml version="1.0" ?>
<C Asunto="Evoluci&#243;n" Caso="AD111163" CentroA="01" CodigoDiagnosticoRelacionado1="N771" CodigoDiagnosticoRelacionado2="D648" CodigoDiagnosticoRelacionado3="" Diagnostico="O234" IdPaciente="117124" Modulo="01" Padre="394469" PlanAdm="100" Prestador="7883096" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&lt;P style=&quot;LINE-HEIGHT: normal; MARGIN: 5pt 0cm; mso-pagination: none; mso-layout-grid-align: none&quot; class=MsoNormal&gt;&lt;SPAN style=&quot;FONT-FAMILY: &#39;Tahoma&#39;,&#39;sans-serif&#39;; COLOR: black; FONT-SIZE: 10pt; mso-bidi-font-size: 12.0pt; mso-fareast-font-family: &#39;Times New Roman&#39;; mso-fareast-language: ES-CO&quot;&gt;PACIENTE FEMENINA DE 16 A&#209;OS CON DIAGNOSTICO DE:&lt;?xml:namespace prefix = o ns = &quot;urn:schemas-microsoft-com:office:office&quot; /&gt;&lt;o:p&gt;&lt;/o:p&gt;&lt;/SPAN&gt;&lt;/P&gt;
&lt;P style=&quot;LINE-HEIGHT: normal; MARGIN: 0cm 0cm 10pt&quot; class

In [87]:
r = registros.loc[393965]

r = xml.dom.minidom.parseString(stringRegistro) #r.RegistroXML)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Epicrisis de la Historia Ingreso de Urgencias" Caso="AD89414" CentroA="01" CodigoDiagnosticoRelacionado1="Z321" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="N771" IdPaciente="106843" Modulo="03" Padre="AD89414" PlanAdm="118" Prestador="73083700" Raiz="01" Registro="450">
	<row NombreCampo="Antecedentes" NombreTabla="CamposTexto" ValorCampo=""/>
	<row NombreCampo="AntecedentesHTML" NombreTabla="CamposTexto" ValorCampo="- PERSONALES:
  1-&amp;amp;gt;LITIASISRENAL
7-&amp;amp;gt;G3P2C0A0
MENARQUIA12A&amp;#209;OS
INICIODEVIDASEXUAL14A&amp;#209;OS
INICIODEVIDAOBSTETRICA15A&amp;#209;OS
COMPA&amp;#209;EROSSEXUALES3
FUM25/2/1514SEMANASHOY
FECHADEULTIMOPARTO9/12/2011

ECOGRAFIA17/4/157SEMNOSEOBSERVAEMBRION
ECOGRAFIA12/5/1510SEMEXTRAPOLADAHOY13.1SEM

NOPARACLINICOSDECONTROLPRENATAL
- FAMILIARES:
  DIABETES - HIPERTENSA MADRE
- AL&amp;#201;RGICOS:
  
- QUIR&amp;#218;RGICOS:"/>
	<row NombreCampo="AplicaCuidadoEnfermeria" NombreTabl

In [88]:
for c in r.childNodes:
    print(c)
    for cc in c.childNodes:
        print(cc, cc.attributes['NombreCampo'].value)

<DOM Element: C at 0x12b45ca50>
<DOM Element: row at 0x12b45c9b0> Antecedentes
<DOM Element: row at 0x12b45c910> AntecedentesHTML
<DOM Element: row at 0x12b45cd70> AplicaCuidadoEnfermeria
<DOM Element: row at 0x12b45c690> AsuntoHistoriaPadre
<DOM Element: row at 0x12b45ce10> Cama
<DOM Element: row at 0x12b45caf0> CambiosDePosicion
<DOM Element: row at 0x12b45ceb0> CodigoDiagnosticoPrincipal
<DOM Element: row at 0x12b45cf50> CodigoDiagnosticoPrincipalSalida
<DOM Element: row at 0x12b45c870> CodigoDiagnosticoRealcionado3
<DOM Element: row at 0x12b17e050> CodigoDiagnosticoRelacionado1
<DOM Element: row at 0x12b17e0f0> CodigoDiagnosticoRelacionado1Salida
<DOM Element: row at 0x12b17e190> CodigoDiagnosticoRelacionado2
<DOM Element: row at 0x12b17e230> CodigoDiagnosticoRelacionado2Salida
<DOM Element: row at 0x12b17e2d0> CodigoDiagnosticoRelacionado3Salida
<DOM Element: row at 0x12b17e370> CodigoHistoriaPadre
<DOM Element: row at 0x12b17e410> CodigoPlanAdministradora
<DOM Element: row at 0x1

In [ ]:
def getDataFromPatient(p):
    """
    Get the patient data
    """
    r = {}
    r['id'] = r.Identificacion
    r['marital'] = p.EstadoCivil
    r['etnia'] = p.Etnia #Map from id to string
    r['estudios'] = p.Escolaridad #Map from id to string
    r['nacimiento'] = p.FechaNac #Map from id to string

In [223]:
rString = registros.RegistroXML[465351]
register = ET.fromstring(rString)

In [236]:
r = xml.dom.minidom.parseString(rString)
print(r.toprettyxml())

<?xml version="1.0" ?>
<C Asunto="Evoluci&amp;#243;n" Caso="AD129559" CentroA="01" CodigoDiagnosticoRelacionado1="Z370" CodigoDiagnosticoRelacionado2="" CodigoDiagnosticoRelacionado3="" Diagnostico="O839" IdPaciente="118111" Modulo="01" Padre="464998" PlanAdm="100" Prestador="9068204" Raiz="94" Registro="106">
	<row NombreCampo="DescripcionNota" NombreTabla="CamposMemo" ValorCampo="&amp;lt;P&amp;gt;PACIENTE DE 20 A&amp;#209;OS CON IDX DE:&amp;lt;BR&amp;gt;1. PARTO VAGINAL ASISTIDO (18/11/15) 8:22 PM&amp;lt;BR&amp;gt;2. RNUV&amp;lt;BR&amp;gt;3. PUERPERIO INMEDIATO&amp;lt;BR&amp;gt;&amp;lt;BR&amp;gt;FC: 78 LPM FR: 17 RPM TA: 110/70 T: 36&amp;lt;BR&amp;gt;&amp;lt;/P&amp;gt;
&amp;lt;P&amp;gt;PACIENTE CONSCIENTE, ALERTA Y ORIENTADA, ENREGULAR ESTADO GENERAL, PALIDEZ GENERALIZADA, PUPILAS ISOC&amp;#211;RICAS NORMOREACTIVAS A LA LUZ, MUCOSAS SECAS Y P&amp;#193;LIDAS, CUELLO SIM&amp;#201;TRICO M&amp;#211;VIL, SIN ADENOPATIAS NI INGURGITACI&amp;#211;N YUGULAR. T&amp;#211;RAX SIMETRICO, EXPANSIB

In [120]:
i = 57
print(i, v[i].attributes['NombreCampo'].value,v[i].nodeValue)
print (" ".join(t.nodeValue for t in v[i].childNodes if t.nodeType == t.TEXT_NODE))

57 Analisis None



In [136]:
from html.parser import HTMLParser
pars =HTMLParser()
pars.unescape(s)

/Users/gbernardino/opt/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The unescape method is deprecated and will be removed in 3.5, use html.unescape() instead.
  This is separate from the ipykernel package so we can avoid doing imports until


'PACIENTE DE 27 AÑOS CON EMBARAZO DE 8 SEMANAS POR FUM + AMENAZA DE ABORTO VS  ABORTO INCOMPLETO, ACTUALMENTE HEMODINAMICAMENTE ESTABLE, AFEBRIL, CON SIGNOS VITALES DENTRO DE PARAMETROS NORMALES, SIN SIGNOS DE SIRS NI IRRITACION PERITONEAL, TOLERANDO OXIGENO AMBIENTE Y VIA ORAL. SE DECIDE INGRESAR A LA PACIENTE PARA REALIZACION DE PARACLINICOS QUE CONFIRMEN DIAGNOSTICO Y REVALORAR.'